## Code to plot offline chemistry

In [124]:
import matplotlib 
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import make_axes_locatable
# plt.ioff()

import numpy as np 
import pickle as pkl 
import glob, json

dir_offline = "/home/n/nichollsh/PROTEUS/output/"

species_cmap = plt.get_cmap('nipy_spectral')
heat_cmap    = plt.get_cmap('gnuplot2_r')

mx_clip_min = 1e-30
mx_clip_max = 1.0

In [96]:
# Function to read PT profile and VULCAN output file and AEOLUS mixing ratios
def read_year(year_int):

    year_data = {}  # Dictionary of mixing ratios, pressure, temperature

    # Read VULCAN output file
    with open("results/%d/output.vul" % year_int, "rb") as handle:
        vul_data = pkl.load(handle)


    # Save year
    year_data["year"] = int(year_int)

    # Parse mixing ratios
    vul_var = vul_data['variable']
    for si,sp in enumerate(vul_var['species']):
        year_data["mx_"+sp] = np.clip(np.array(vul_var['ymix'])[:,si],mx_clip_min,mx_clip_max)

    # Parse PT profile
    vul_atm = vul_data['atm']
    year_data["pressure"] =     np.array(vul_atm["pco"]) / 1e6  # convert to bar
    year_data["temperature"] =  np.array(vul_atm["Tco"])

    # Read AEOLUS mixing ratios which were used to initialise VULCAN
    vol_data_str = None
    with open("results/%d/vulcan_cfg.py" % year_int, "r") as f:
        lines = f.read().splitlines()
        for ln in lines:
            if "const_mix" in ln:
                vol_data_str = ln.split("=")[1].replace("'",'"')

    if vol_data_str == None:
        raise Exception("Could not parse vulcan cfg file!")
    
    vol_data = json.loads(vol_data_str)
    for vol in vol_data:
        ae_mx = float(vol_data[vol])
        if ae_mx > 1e-12:
            year_data["ae_"+vol] = ae_mx

    return year_data


In [97]:
# Plot mixing ratios and PT profile for a given year
def plot_year(year_dict: dict, species: list):
    lw = 2

    fig, (ax0,ax1) = plt.subplots(1,2,sharey=True,figsize=(8,4))

    # Temperature profile
    ax0.set_title("Time = %g kyr" % (year_dict["year"]/1.e3))
    ax0.set_ylabel("Pressure [bar]")
    ax0.invert_yaxis()
    ax0.set_yscale("log")
    ax0.set_xlabel("Temperature [K]")
   
    ax0.plot(year_dict["temperature"],year_dict["pressure"],color='black',lw=lw)

    # Mixing ratios
    ax1.set_xlabel("Mixing ratio")
    ax1.set_xscale("log")


    min_mix = 5e-1
    for i,s in enumerate(species):

        color = species_cmap(float(i+1)/len(species))

        # VULCAN result
        key = str("mx_"+s)
        if key in year_dict.keys():
            ax1.plot(year_dict[key],year_dict["pressure"],lw=lw+0.4,color='black')
            ax1.plot(year_dict[key],year_dict["pressure"],label=s,lw=lw,color=color)
            min_mix = min(min_mix,np.amin(year_dict[key]))
            
        # AEOLUS result
        key = str("ae_"+s)
        if key in year_dict.keys():
            ax1.plot(np.ones((len(year_dict["pressure"]))) * year_dict[key],year_dict["pressure"],linestyle='--',lw=lw*0.5,color=color)
            
    ax1.set_xlim([min_mix,1])
    ax1.legend(loc="lower left")

    fig.tight_layout()
    fig.savefig("plots/year_%d.pdf"%year_dict["year"])
    plt.close(fig)
    

In [120]:
# Plot time-evolution of PT profile and a single species for all years
def plot_species(yrs, sp: str, tmin=-1.0, tmax=-1.0):

    lw = 2
    alpha = 0.7

    fig, (ax0,ax1) = plt.subplots(1,2,sharey=True,figsize=(8,4))

    ax0.set_ylabel("Pressure [bar]")
    ax0.invert_yaxis()
    ax0.set_yscale("log")
    ax0.set_xlabel("Temperature [K]")
    ax0.set_title("Evolution of $T(p)$")

    ax1.set_title("Evolution of "+sp)
    ax1.set_xlabel("Mixing ratio")
    ax1.set_xscale("log")

    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size='5%', pad=0.05)

    if (tmin > 0):
        cb_vmin = tmin
    else:
        cb_vmin = max(yrs[0]["year"],1.0)
        
    
    if (tmax > 0):
        cb_vmax = tmax
    else:
        cb_vmax = yrs[-1]["year"]
        

    norm = matplotlib.colors.LogNorm(vmin=cb_vmin, vmax=cb_vmax)
    sm = plt.cm.ScalarMappable(cmap=heat_cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, cax=cax, orientation='vertical')  #, ticks=np.linspace(0,2,N), boundaries=np.arange(-0.05,2.1,.1))
    cbar.set_label("Time [yr]") 
    
    min_mix = 5e-1
    
    for i, yd in enumerate(yrs):

        if (tmin > -1) and (yd["year"] < tmin):
            continue 
        
        if (tmax > -1) and (yd["year"] > tmax):
            continue 

        color = sm.to_rgba(yd["year"])

        p = yd["pressure"]

        # Temperature profile
        ax0.plot(yd["temperature"],p,color=color,lw=lw,alpha=alpha,label="%1.2e"%yd["year"])

        # Mixing ratios
        key = str("mx_"+sp)
        ax1.plot(yd[key],p,label=sp,color=color,lw=lw,alpha=alpha)

        min_mix = min(min_mix,np.amin(yd[key]))

        key = str("ae_"+sp)
        if key in yd.keys():
            ax1.plot(np.ones((len(p),)) * yd[key],p,linestyle='--',lw=lw*0.5,color=color)
            
    ax1.set_xlim([min_mix,1])

    fig.tight_layout()
    fig.savefig("plots/species_%s.pdf"%sp)
    plt.close(fig)
    

In [130]:
# Plot of evolution of mean mixing ratios over time
def plot_evolution(yrs, species: list):

    lw = 2

    fig, ax = plt.subplots(1,1,figsize=(8,4))

    ax.set_ylabel("Mean mixing ratio")
    ax.set_yscale("log")
    ax.set_xlabel("Time")
    ax.set_title("Evolution of mean mixing ratio over time")
    ax.set_xlabel("Time [yr]")
    ax.set_xscale("log")

    for i,sp in enumerate(species):
        color = species_cmap(float(i+1)/len(species))

        times =     []
        mx_vul =    []
        mx_aeo =    []

        for yd in yrs:
            times.append(yd["year"])

            clean_mx = np.nan_to_num(yd["mx_"+sp])
            mean_mx = np.mean(clean_mx)
            mx_vul.append(float(mean_mx))

        ax.plot(times,mx_vul,color='black',lw=lw+0.4)
        ax.plot(times,mx_vul,color=color,label=sp,lw=lw)

        if str("ae_"+sp) in yrs[0].keys():
            for yd in yrs:
                clean_mx = np.nan_to_num(yd["ae_"+sp])
                mean_mx = np.mean(clean_mx)
                mx_aeo.append(float(mean_mx))

            ax.plot(times,mx_aeo,color=color,linestyle='--',lw=lw)


    ax.legend(loc="lower left",ncol=6)
    ax.set_ylim([1e-20,1.5])

    ax.set_xlim([max(times[0],1.0),times[-1]*1.2])

    ax.axvline(x=times[-1],color='black',lw=0.7)

    fig.tight_layout()
    fig.savefig("plots/evolution.pdf")
    plt.close(fig)
    
    

### Do it

In [100]:
ls = glob.glob("results/*/output.vul")
years = [int(f.split("/")[-2]) for f in ls]
years_data = [read_year(y) for y in years]
print(years)
print(years_data[0].keys())

if len(years) == 0:
    raise Exception('No VULCAN output files found')

[0, 1, 2, 3, 6, 10, 28, 40, 64, 100, 150, 250, 450, 700, 1000, 1619, 2703, 4453, 7125, 11234, 17727, 28935, 45734]
dict_keys(['year', 'mx_OH', 'mx_H2', 'mx_H2O', 'mx_H', 'mx_O', 'mx_CH', 'mx_C', 'mx_CH2', 'mx_CH3', 'mx_CH4', 'mx_C2', 'mx_C2H2', 'mx_C2H', 'mx_C2H3', 'mx_C2H4', 'mx_C2H5', 'mx_C2H6', 'mx_CO', 'mx_CO2', 'mx_CH2OH', 'mx_H2CO', 'mx_HCO', 'mx_CH3O', 'mx_CH3OH', 'mx_CH3CO', 'mx_O2', 'mx_H2CCO', 'mx_HCCO', 'mx_N', 'mx_NH', 'mx_CN', 'mx_HCN', 'mx_NO', 'mx_NH2', 'mx_N2', 'mx_NH3', 'mx_HO2', 'mx_N2H2', 'mx_N2H', 'mx_N2H3', 'mx_N2H4', 'mx_HNO', 'mx_H2CN', 'mx_HNCO', 'mx_NO2', 'mx_N2O', 'mx_C4H2', 'mx_CH2NH2', 'mx_CH2NH', 'mx_CH3NH2', 'mx_CH3CHO', 'mx_C3H4', 'mx_HNO2', 'mx_NCO', 'mx_O3', 'mx_H2O2', 'mx_NO3', 'mx_HC3N', 'mx_CH3CN', 'mx_CH2CN', 'mx_C2H3CN', 'mx_C3H3', 'mx_C3H2', 'mx_C4H3', 'mx_C4H5', 'mx_C6H6', 'mx_CH2_1', 'mx_HNO3', 'mx_N2O5', 'mx_C6H5', 'mx_O_1', 'mx_N_2D', 'mx_He', 'mx_CH3O2', 'pressure', 'temperature', 'ae_H2O', 'ae_CO2', 'ae_H2', 'ae_N2', 'ae_CH4', 'ae_O2', 'ae_C

In [101]:
# Species to plot
species = ["H2", "H2O", "H", "OH", "CO2", "CO", "CH4","HCN", "NH3", "N2", "NO"]

In [131]:
# Plot evolution of mixing ratios
print("Plotting evolution")
plot_evolution(years_data, species)

Plotting evolution


In [103]:
# Make plots for each year
print("Plotting PT and MX")
for yd in years_data:
    plot_year(yd, species=species)

Plotting PT and MX


In [132]:
# Make species plots
print("Plotting species")
for s in species:
    plot_species(years_data,s, tmin=1e1)


Plotting species


In [ ]:
print("Done")

Done
